In [2]:
import heapq #import biblioteki pozwalającej na skorzystanie ze stosu
import string 
import pickle #zapis i odczyt z pliku
import math
from bitarray import bitarray 

# struktura drzewa
#klasa wierzchołek w drzewie
codes = {}
reverse_code = {}
class Node:
    def __init__(self, letter, probability):
        self.left = None
        self.right = None
        self.letter = letter # jaka to litera
        self.probability = probability # prawdopodobieństwo wystąpienia
    #porównanie
    def __lt__(self, other):
        if(other == None):
            return -1
        if(not isinstance(other, Node)):
            return -1
        return self.probability < other.probability
#określenie prawdopodobieństwa wystąpienie poszczególnego znaku w tekscie
def probability(text):
    wynik = {}
    for letter in text:
        if not letter in wynik:
                wynik[letter] = 0
        wynik[letter] += 1
    for i in wynik:
        wynik[i] = wynik[i]/len(text)
    return wynik
#utworzenie stosu wierzchołków
def make_heap(frequency):
    heap=[]
    for key in frequency:
        node = Node(key, frequency[key])
        heapq.heappush(heap, node)
    return heap
#połączenie wierzchołków w drzewo, zwraca jeden element który
def merge_nodes(heap):
    while(len(heap)>1):
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)

        merged = Node(None, node1.probability + node2.probability)
        merged.left = node1
        merged.right = node2

        heapq.heappush(heap, merged)
    return heap

#tworzenie ścieżki w drzewie ustalającej kod znaku
def make_codes_helper(root, current_code):
   
    if(root == None):
        return

    if(root.letter != None):
        codes[root.letter] = current_code
        #tworzenie odwrotnego słownika zgodnie z sugestią
        reverse_code[current_code] = root.letter
        return

    make_codes_helper(root.left, current_code + "1")
    make_codes_helper(root.right, current_code + "0")
    

#kolejno "ściąganie" elementów ze stosu oraz zapisanie gotowego kodu do pliku
def make_codes(heap):
    root = heapq.heappop(heap)
    current_code = ""
    make_codes_helper(root, current_code)
    with open("huffman.txt", 'wb') as handle:
      pickle.dump(reverse_code, handle)
    


#kodowanie tekstu
def code_text(sample_text):
    crypted_text = ""
    for char in sample_text:
        crypted_text += codes[char]
    return crypted_text

#porównanie stringów
def compare(original_text, decoded_text):
    if original_text == decoded_text:
        print('Tekst zgodny z oryginałem')
    else:
        print('Występują błędy')
        
#odczyt pliku textowego          
def reading_data(filetextname):
    f=open(filetextname, "r")
    contents = f.read()
    return contents
#otworzenie bitarray oraz kodu z plkiów
def open_text():
    with open('coded_text.txt', 'rb') as handle:
      loaded = pickle.loads(handle.read())
    with open("huffman.txt", 'rb') as handle:
        slownik = pickle.loads(handle.read())
    return loaded, slownik   
#przekonwertowanie bitarray do stringa
def bitarray_to_str(bit_arr):
    value = ""
    for i in bit_arr:
        if i==0:
            value += '0'
        else:
            value += '1'
    return value
#zapisanie tekstu do pliku
def save_text(coded_text):
    bit_text = bitarray()
    bit_text.extend(coded_text)
    with open('coded_text.txt', 'wb') as handle:
          pickle.dump(bit_text, handle)
            
#dekodowanie tekstu - każda litera ma oryginalny kod, nie ma potrzeby wyrównywać bitów
def decode_text(crypted_text,rev_dict):
    decoded = ""
    symbol = ""
    for i in range(0,len(crypted_text)):
        symbol+=crypted_text[i]
        if symbol in rev_dict:
            decoded += rev_dict[symbol]
            symbol =""
    return decoded
#liczenie entropii po słowniku prawdopodobieństw
def entropy(prob):
    final = 0
    for i in prob:
            final +=prob[i]*math.log(prob[i],2)
    return -final
#średnia długość kodu 
def avg_len(codes,prob):
    result = 0
    for i in codes:
        result+=len(codes[i])*prob[i]
    return result
#efektywność
def effiency(codes, prob):
    if avg_len(codes,prob)==0:
        return 0
    else:
        return entropy(prob)/avg_len(codes,prob)

In [14]:
#czyszczenie globalnych słowników
reverse_code = {}
codes ={}
#odczyt danych z pliku
sample = reading_data("norm_wiki_sample.txt")
# sample = "abbcccdddd"
#utworzenie słownika prawdopodobieństw wystąpienia znaku na podstawie tekstu
slownik = probability(sample)
#tworzenie stosu
stos = make_heap(slownik)
#tworzenie drzewa
tree = merge_nodes(stos)
#tworzenie kodu
make_codes(tree)
#szyfrowanie tekstu
crypted=code_text(sample)
#zapis zakodowanego tekstu do pliku
save_text(crypted)
#pobranie zakodowanego tekstu oraz kodu
barray_text,  cypher = open_text()
#dekodowanie tekstu
result = decode_text(bitarray_to_str(barray_text),cypher)
#porównianie oryginału z tekstem pod dekodowaniu
compare(sample,result)
#wyliczenie efektywnosci kodowania
effiency(codes, slownik)

Tekst zgodny z oryginałem


0.9933582848516693